<a href="https://colab.research.google.com/github/canerskrc/GraphRAG/blob/main/GraphRAG_Dependency_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx
from transformers import AutoTokenizer, AutoModelForCausalLM

# Step 1: Create a graph to model software dependencies
dependency_graph = nx.DiGraph()

# Add nodes (libraries, versions, features)
dependency_graph.add_node("Django", version="4.0", description="A Python web framework for rapid development.")
dependency_graph.add_node("Django", version="3.2", description="An older version of Django with long-term support (LTS).")
dependency_graph.add_node("DRF", version="3.12", description="Django REST Framework for building REST APIs.")
dependency_graph.add_node("DRF", version="3.14", description="Latest DRF version with advanced features.")
dependency_graph.add_node("Python", version="3.8", description="Python programming language version 3.8.")
dependency_graph.add_node("Python", version="3.10", description="Python programming language version 3.10.")

# Add edges to represent compatibility and dependencies
dependency_graph.add_edges_from([
    ("Django 4.0", "Python 3.10", {"type": "requires"}),
    ("Django 3.2", "Python 3.8", {"type": "requires"}),
    ("DRF 3.14", "Django 4.0", {"type": "requires"}),
    ("DRF 3.12", "Django 3.2", {"type": "requires"})
])

# Step 2: Retrieve relevant nodes based on query
def retrieve_relevant_nodes(query, graph):
    relevant_nodes = []
    for node, data in graph.nodes(data=True):
        if query.lower() in str(node).lower() or query.lower() in str(data.get('description', '')).lower():
            relevant_nodes.append((node, data))
    return relevant_nodes

# Step 3: Generate chatbot response using relevant nodes
def generate_response(query, retrieved_nodes):
    context = f"Query: {query}\nRelevant Information:\n"
    for node, data in retrieved_nodes:
        description = data.get("description", "No description available.")
        context += f"{node}: {description}\n"

    # Load pre-trained language model
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Generate response
    input_ids = tokenizer.encode(context, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=200, num_return_sequences=1)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Step 4: Chatbot function to integrate retrieval and generation
def chatbot(query):
    retrieved_nodes = retrieve_relevant_nodes(query, dependency_graph)
    if retrieved_nodes:
        response = generate_response(query, retrieved_nodes)
        return response
    else:
        return "I couldn't find any relevant information for your query. Please refine your question."

# Example interaction
user_query = "How can I resolve a conflict between Django 4.0 and DRF 3.12?"
response = chatbot(user_query)

# Display chatbot's response
print("User Query:", user_query)
print("Chatbot Response:", response)
